# Problems

## Problem 1

The following function:

- downloads all hourly data for the previous five days for the five FAANG stocks.

   https://www.investopedia.com/terms/f/faang-stocks.asp

- saves the data into a folder called "data" in the root of this repository using a filename with the format "YYYYMMDD-HHmmss.csv" where "YYYYMMDD" is the four-digit year (e.g. 2025), followed by the two-digit month (e.g. 09 for September), followed by the two digit day, and "HHmmss" is hour, minutes, seconds. 

In [6]:
# Import required libraries.

# Dates and times.
import datetime as dt # For handling dates and times. 
# Data frames.
import pandas as pd # For data manipulation and analysis. 
# Yahoo Finance data.
import yfinance as yf # For downloading financial data from Yahoo Finance. 

In [7]:
def get_data(days=5, interval='1h', folder='./data/'):
    
    # Define the list of FAANG stocks.
    FAANG = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG']
    
    # Get the current date and time .
    now = dt.datetime.now()
    
    # Download hourly data for each FAANG stock.
    df = yf.download(FAANG, period=f'{days}d', interval=interval, auto_adjust=True)
    
    # Generate the filename with the current date and time.
    filename = folder + now.strftime("%Y%m%d-%H%M%S") + ".csv"
    
    # Save the data to a CSV file.
    df.to_csv(filename)

    return filename

## Problem 2

## Problem 3